In [19]:
import boto3
import time
import zipfile
from io import BytesIO
import sagemaker

In [20]:
# 630 MB dataset
#DATASET_SOURCE = {
#    'Bucket': 'aws-jam-challenge-resources',
#    'Key': 'covid-19-with-sage-maker/radiography_train_data.zip'
#}
#LOCAL_ZIP = 'dataset/radiography_train_data.zip'

# 3 MB dataset
DATASET_SOURCE = {
    'Bucket': 'aws-jam-challenge-resources',
    'Key': 'covid-19-with-sage-maker/radiography_train_data_10.zip'
}
LOCAL_ZIP = 'dataset/radiography_train_data_10.zip'

### Creating default S3 Bucket in the JAM Account

In [21]:
# get the region and sagemaker session
region = boto3.Session().region_name
session = sagemaker.Session()

# create a sagemaker default bucket
BUCKET = session.default_bucket() 
print ("Created Default Bucket: ", BUCKET)

s3_resource = boto3.resource('s3')
jam_bucket = s3_resource.Bucket(BUCKET)

Created Default Bucket:  sagemaker-eu-central-1-154108359553


### Creating local S3 Bucket in the JAM Account

In [22]:
'''
BUCKET = "covid-19-image-dataset-"+str(int(time.time()))
REGION = "us-west-1"
s3_client = boto3.client('s3', region_name=REGION)
s3_resource = boto3.resource('s3')
jam_bucket = s3_resource.Bucket(BUCKET)

print ("Creating Bucket: ", BUCKET)
s3_client.create_bucket(Bucket=BUCKET,
                        CreateBucketConfiguration={'LocationConstraint': REGION})
'''

'\nBUCKET = "covid-19-image-dataset-"+str(int(time.time()))\nREGION = "us-west-1"\ns3_client = boto3.client(\'s3\', region_name=REGION)\ns3_resource = boto3.resource(\'s3\')\njam_bucket = s3_resource.Bucket(BUCKET)\n\nprint ("Creating Bucket: ", BUCKET)\ns3_client.create_bucket(Bucket=BUCKET,\n                        CreateBucketConfiguration={\'LocationConstraint\': REGION})\n'

### Copying zipped Dataset into the local S3 Bucket

In [23]:
print ("Copying Data from: ", DATASET_SOURCE)
jam_bucket.copy(DATASET_SOURCE, LOCAL_ZIP)

Copying Data from:  {'Bucket': 'aws-jam-challenge-resources', 'Key': 'covid-19-with-sage-maker/radiography_train_data_10.zip'}


### Unzipping Dataset into the local S3 Bucket

In [24]:
print ("Unzipping ", LOCAL_ZIP)
zip_obj = s3_resource.Object(bucket_name=BUCKET, key=LOCAL_ZIP)
buffer = BytesIO(zip_obj.get()["Body"].read())
z = zipfile.ZipFile(buffer)
for filename in z.namelist():
    file_info = z.getinfo(filename)
    s3_resource.meta.client.upload_fileobj(
        z.open(filename),
        Bucket=BUCKET,
        Key=f'{filename}'
    )
print ("Completed Unzipping Training Data")
jam_bucket.objects.filter(Prefix="dataset/").delete()

Unzipping  dataset/radiography_train_data_10.zip
Completed Unzipping Training Data


[{'ResponseMetadata': {'RequestId': 'KGF60R3B05Q0VVGS',
   'HostId': 'H+DpBsfE32uRfdkhJdV4AYRNeJY00slbXpL4VZ6pCE4tGAuCbiTSD2nInpyKa/5yRCrzH+LImpM=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'H+DpBsfE32uRfdkhJdV4AYRNeJY00slbXpL4VZ6pCE4tGAuCbiTSD2nInpyKa/5yRCrzH+LImpM=',
    'x-amz-request-id': 'KGF60R3B05Q0VVGS',
    'date': 'Tue, 02 Nov 2021 16:54:10 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'dataset/radiography_train_data_10.zip'}]}]